In [2]:
import os
import cv2
import numpy as np
import mediapipe as mp
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight

In [3]:
DATA_DIR = 'hand_gestures'

In [5]:
# === Load only valid class folders ===
LABELS = sorted([d for d in os.listdir(DATA_DIR) if os.path.isdir(os.path.join(DATA_DIR, d))])
LABEL_TO_INDEX = {label: idx for idx, label in enumerate(LABELS)}
NUM_CLASSES = len(LABELS)

In [6]:
NUM_CLASSES

17

In [7]:
mp_hands = mp.solutions.hands

In [8]:
def extract_landmarks(image_path):
    image = cv2.imread(image_path)
    if image is None:
        return None
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    with mp_hands.Hands(static_image_mode=True) as hands:
        result = hands.process(image_rgb)
        if result.multi_hand_landmarks:
            landmarks = []
            for lm in result.multi_hand_landmarks[0].landmark:
                landmarks.extend([lm.x, lm.y, lm.z])
            return landmarks
    return None

In [9]:
def load_dataset():
    X, y = [], []
    for label in LABELS:
        label_dir = os.path.join(DATA_DIR, label)
        print(f"Loading class '{label}' with index {LABEL_TO_INDEX[label]}")
        for img_name in os.listdir(label_dir):
            img_path = os.path.join(label_dir, img_name)
            try:
                landmarks = extract_landmarks(img_path)
                if landmarks:
                    X.append(landmarks)
                    y.append(LABEL_TO_INDEX[label])
            except Exception as e:
                print(f"Error processing {img_path}: {e}")
    return np.array(X), to_categorical(y, NUM_CLASSES)

In [10]:
print("Loading data...")
X, y = load_dataset()
print(f"Loaded {len(X)} samples.")

Loading data...
Loading class 'Cool' with index 0
Loading class 'Good Bye' with index 1
Loading class 'Good Luck' with index 2
Loading class 'Got it' with index 3
Loading class 'Hi' with index 4
Loading class 'I Love You' with index 5
Loading class 'Love' with index 6
Loading class 'Okay' with index 7
Loading class 'Perfect' with index 8
Loading class 'Salute' with index 9
Loading class 'Stop' with index 10
Loading class 'Strong' with index 11
Loading class 'Thumbs up' with index 12
Loading class 'Victory' with index 13
Loading class 'Wait' with index 14
Loading class 'Wait a minute' with index 15
Loading class 'Walk' with index 16
Loaded 603 samples.


In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:
y_int = np.argmax(y_train, axis=1)
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_int), y=y_int)
class_weights_dict = dict(enumerate(class_weights))


In [13]:
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(NUM_CLASSES, activation='softmax')
])

c:\users\karan\appdata\local\programs\python\python39\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [14]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [16]:
print("Training model...")
model.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test), class_weight=class_weights_dict)

Training model...
Epoch 1/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7340 - loss: 0.8011 - val_accuracy: 0.8099 - val_loss: 0.7106
Epoch 2/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7398 - loss: 0.7488 - val_accuracy: 0.8017 - val_loss: 0.6826
Epoch 3/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7478 - loss: 0.7767 - val_accuracy: 0.7934 - val_loss: 0.6494
Epoch 4/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7461 - loss: 0.7442 - val_accuracy: 0.7769 - val_loss: 0.7139
Epoch 5/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7870 - loss: 0.6640 - val_accuracy: 0.8017 - val_loss: 0.6267
Epoch 6/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7658 - loss: 0.7061 - val_accuracy: 0.7686 - val_loss: 0.6721
Epoch 7/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7677 - loss: 0.6586 - val_accuracy: 0.7934 - val_loss: 0.6399
Epoch 8/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7631 - loss: 0.7166 - val_a

In [17]:
model.save('word_gesture_model.h5')
print("Model trained and saved successfully.")

Model trained and saved successfully.
